In [ ]:
# Import the required libraries
import ee
import geemap
import datetime
import calendar
import math
import os

In [ ]:
# Initialize geemap
Map = geemap.Map()

In [ ]:
# South and North Gojjam sub-basins renamed as Choke
choke = ee.FeatureCollection("projects/ee-yilkalg3/assets/Choke_watersheds/Choke")

# Add the Choke sub-basins to the map
Map.addLayer(choke, {}, "Choke")
Map.centerObject(choke, 8)
Map

In [ ]:
# Area of Choke in sqkm
area = choke.geometry().area().divide(1000000).round()
print('Area of Choke in Sq km:', area.getInfo())

## R (Erosivity)


In [ ]:
# Define the start and end years for rainfall calculation.
start_year = 1983
end_year = 2024

In [ ]:
years = ee.List.sequence(start_year, end_year)
def get_annual_rainfall(year):
    start = ee.Date.fromYMD(year, 1, 1)
    end = ee.Date.fromYMD(year, 12, 31)
    chirps = ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY').filterDate(start, end)
    return chirps.select('precipitation').sum().clip(choke).set('year', year)

annual_rainfall_collection = ee.ImageCollection(years.map(get_annual_rainfall))
annual_rainfall = annual_rainfall_collection.mean()

In [ ]:
# Download the annual rainfall
#geemap.ee_export_image(annual_rainfall, filename='Rainfall.tif', scale=5000, region=choke.geometry(), file_per_band=False)

In [ ]:
# minimum and maximum.
stats = annual_rainfall.reduceRegion(reducer=ee.Reducer.minMax(), geometry=choke, scale=5000)
print('Min and Max:', stats.getInfo())

# Add the annual rainfall to the map. 
"""
Map.addLayer(annual_rainfall, {
        'min': 800,
        'max': 1500,
        'palette': ['white', 'green', 'blue']
}, 'Annual Rainfall')

"""
# calculate the mean and standard deviation of annual rainfall
mean_rainfall = annual_rainfall.reduceRegion(reducer=ee.Reducer.mean(), geometry=choke, scale=5000)
print('Mean:', mean_rainfall.getInfo())

In [ ]:
# R factor (Juliette 2007)
r_factor = ee.Image().expression(
  '47.6 + 0.36 * P',
  {
    'P': annual_rainfall.select('precipitation')
  }).rename('R_factor')

# Reduce the image within the region of interest to its minimum and maximum.
stats = r_factor.reduceRegion(reducer=ee.Reducer.minMax(), geometry=choke, scale=5000)
print('Min and Max:', stats.getInfo())

# calculate the mean and standard deviation of annual rainfall
mean_r = r_factor.reduceRegion(reducer=ee.Reducer.mean(), geometry=choke, scale=5000)
print('Mean:', mean_r.getInfo())

# Add the R factor to the map.
Map.addLayer(r_factor, {
        'min': 300,
        'max': 800,
        'palette': ['white', 'yellow', 'red']
}, 'R Factor')

In [ ]:
# Download the annual rainfall
#geemap.ee_export_image(r_factor, filename='R_Factor.tif', scale=90, region=choke.geometry(), file_per_band=False)

## K (Erodibilty)


In [ ]:
# Soil Data 
soil = ee.FeatureCollection("projects/ee-yilkalg3/assets/gojjam_soil")

# Aggregate the unique values of the 'domRSG' property
domRSG = soil.aggregate_array('SOIL_TYPE').distinct().getInfo()
#print('domRSG:', domRSG)

# Print the area of each unique value
for value in domRSG:
    area_value = soil.filter(ee.Filter.eq('SOIL_TYPE', value)).geometry().area().divide(1000000).getInfo()
    #print(f"Area of {value}: {area_value} sq km")

In [ ]:
# Define K-factor values as a dictionary
k_values = {
    "calcaric flubisols": 0.3,
    "calcic cambisols": 0.2,
    "calcic fluvisols": 0.3,
    "calcic xerosols": 0.3,
    "chromic cambisols": 0.23,
    "chromic luvisols": 0.4,
    "chromic vertisols": 0.19,
    "dystric fluvisols": 0.3,
    "dystric gleysols": 0.29,
    "dystric nitosols": 0.25,
    "eutric cambisols": 0.35,
    "eutric fluvisols": 0.33,
    "eutric nitisols": 0.25,
    "eutric regosols": 0.2,
    "eutric vertisols": 0.14,
    "leptosols": 0.35,
    "orthic acrisols": 0.33,
    "orthic luvisols": 0.2,
    "orthic solonchaks": 0.17,
    "pellic vertisols": 0.22,
    "phaeozems": 0.2,
    "vertic cambisols": 0.3
}

# Function to assign K-factor based on soil type
def assign_k_factor(feature):
    soil_type = feature.get("SOIL_TYPE")  # Get the soil type from the feature
    k_factor = ee.Dictionary(k_values).get(soil_type, 0)
    return feature.set("K_factor", k_factor)

# Apply function to feature collection
soil = soil.map(assign_k_factor)

# Convert to raster
k_factor = soil.reduceToImage(["K_factor"], ee.Reducer.last())
"""
# Define visualization parameters
k_vis = {'min': 0.15, 'max': 0.33, 'palette': ['green', 'yellow', 'orange', 'red']}

# Add K-factor to the map
Map.addLayer(k_factor, k_vis, "K Factor")
Map
"""

In [ ]:
# Reduce the image within the region of interest to its minimum and maximum.
stats = k_factor.reduceRegion(reducer=ee.Reducer.minMax(), geometry=choke, scale=90)
print('Min and Max:', stats.getInfo())

# calculate the mean and standard deviation of annual rainfall
mean_k = k_factor.reduceRegion(reducer=ee.Reducer.mean(), geometry=choke, scale=90)
print('Mean:', mean_k.getInfo())

In [ ]:
# Download the annual rainfall
#geemap.ee_export_image(k_factor, filename='K_Factor.tif', scale=90, region=choke.geometry(), file_per_band=False)

## LS (Topography)


In [ ]:
dem = ee.Image('CGIAR/SRTM90_V4').select('elevation').clip(choke)
slope = ee.Terrain.slope(dem).clip(choke)
slope = slope.multiply(3.14).divide(180)
flow_accumulation = ee.ImageCollection("projects/sat-io/open-datasets/HYDROGRAPHY90/base-network-layers/flow_accumulation")
flow_accumulation = flow_accumulation.mosaic().select('b1').clip(choke)

In [ ]:
ls_factor = (flow_accumulation.multiply(1000)).divide(22.13).pow(0.4)\
            .multiply(slope.sin().divide(0.0896).pow(1.3)).rename('LS_factor')

# Ls fatcor stats
ls_stats = ls_factor.reduceRegion(reducer=ee.Reducer.minMax(), geometry=choke, scale=90)
print('Min and Max of LS factor:', ls_stats.getInfo())

ls_mean = ls_factor.reduceRegion(reducer=ee.Reducer.mean(), geometry=choke, scale=90)
print('Mean of LS factor:', ls_mean.getInfo())

# Add ls factor to the map
Map.addLayer(ls_factor, {
  'min': 0,
  'max': 300,
  'palette': ['white', 'blue', 'red']
}, 'LS Factor')

Map

In [ ]:
# Download the LS
#geemap.ee_export_image(ls_factor, filename='LS_Factor.tif', scale=90, region=choke.geometry(), file_per_band=False)

## C (Cover)


In [ ]:
# set the date range for the dynamic world land cover data
start = ee.Date('2024-01-01')
end = ee.Date('2024-12-31')

# Dynamic World land cover 
landcover = geemap.dynamic_world(choke, start, end, clip=True,
                                 return_type="class")

Map.addLayer(landcover.randomVisualizer(), {}, 'Land Cover')

In [ ]:
# Change the values of each class by adding 1 to each class to avoid zero values
#landcover = landcover.add(1).toByte()
# Download the landcover
#geemap.ee_export_image(landcover, filename='landcover.tif', scale=90, region=choke.geometry(), file_per_band=False)

In [ ]:
# Codes and Classes (0 = Water, 1 = Forest, 2 = Grass, 3 = Wetland, 4 = Crop, 5 = Shrub, 6 = Built-up, 7 = Bare)
water = landcover.eq(0).multiply(0)
forest = landcover.eq(1).multiply(0.01) # Hurni 1985
grass = landcover.eq(2).multiply(0.05)  # Hurni 1985
wetland = landcover.eq(3).multiply(0.01)    # Bewket 2009
crop = landcover.eq(4).multiply(0.20)       # Hurni 1985 and Bewket 2009 (Average of 0.15 and 0.25)
shrub = landcover.eq(5).multiply(0.05)      # Hurni 1985 as degraded grass
built = landcover.eq(6).multiply(0.01)      # Haregeweyn 2017
bare = landcover.eq(7).multiply(0.4)        # Hurni 1985

In [ ]:
# Combine the land cover classes
c_factor = water.add(forest).add(grass).add(wetland).add(crop).add(shrub).add(built).add(bare)

# Add land cover to the map with random colors
Map.addLayer(c_factor.randomVisualizer(), {}, 'C Factor')

In [ ]:
# Download the C Factor
#geemap.ee_export_image(c_factor, filename='c_factor.tif', scale=90, region=choke.geometry(), file_per_band=False)

## P (Management)


In [ ]:
# Terrrace data
tnt = ee.Image("projects/ee-yilkalg3/assets/TNT")

# Print the unique values in the TNT image
unique_values = tnt.reduceRegion(reducer=ee.Reducer.frequencyHistogram(), geometry=choke, scale=1000)

print('Unique values:', unique_values.getInfo())

In [ ]:
# P Factor value based on the land cover and Terracing
p_factor = ee.Image().expression(
    """ 
    (tnt == 3 && landcover == 2) ? 0.4 :
    (tnt == 3 && landcover == 4) ? 0.32 :
    (tnt == 3 && landcover == 7) ? 0.4 :
    (landcover == 2) ? 0.8 :
    (landcover == 3) ? 0.4 :
    (landcover == 5) ? 0.8 :
    (landcover == 6) ? 0.9 :
    (landcover == 7) ? 0.9 : 1
    """,
    {
        'tnt': tnt,
        'landcover': landcover
    }).rename('P_factor')

# Statistics of P factor
p_stats = p_factor.reduceRegion(reducer=ee.Reducer.minMax(), geometry=choke, scale=90)
#print('Min and Max of P factor:', p_stats.getInfo())

In [ ]:
# Download the C Factor
#geemap.ee_export_image(p_factor, filename='p_factor.tif', scale=90, region=choke.geometry(), file_per_band=False)

## A (Estimate soil loss)


In [ ]:
soil_loss = r_factor.multiply(k_factor).multiply(ls_factor).multiply(c_factor).multiply(p_factor).rename("Soil Loss")

# Soil loss stats
soil_loss_stats = soil_loss.reduceRegion(reducer=ee.Reducer.minMax(), geometry=choke, scale=90)
#print('Min and Max of Soil Loss:', soil_loss_stats.getInfo())

soil_loss_mean = soil_loss.reduceRegion(reducer=ee.Reducer.mean(), geometry=choke, scale=90)
#print('Mean Soil Loss:', soil_loss_mean.getInfo())

In [ ]:
# Assign soil loss greater than 300 to 300
soil_loss = soil_loss.where(soil_loss.gt(300), 300)

# Soil loss stats
#soil_loss_stats = soil_loss.reduceRegion(reducer=ee.Reducer.minMax(), geometry=choke, scale=90)
#print('Min and Max of Soil Loss:', soil_loss_stats.getInfo())

# Mean soil loss
soil_loss_mean = soil_loss.reduceRegion(reducer=ee.Reducer.mean(), geometry=choke, scale=90)
#print('Mean Soil Loss:', soil_loss_mean.getInfo())

In [ ]:
# Prepare a mask with flow accumulation greater than 5
gully_mask = flow_accumulation.gt(450).rename('gully_mask')

# Assign the gully mask soil loss of 5 t/ha/year
soil_loss_with_gully = soil_loss.where(gully_mask, 5)

# Mean soil loss
total_soil_loss_mean = soil_loss_with_gully.reduceRegion(reducer=ee.Reducer.mean(), geometry=choke, scale=90)
print('Mean Soil Loss:', total_soil_loss_mean.getInfo())

In [ ]:
# Add total soil loss to the map
Map.addLayer(soil_loss_with_gully, {
  'min': 0,
  'max': 300,
  'palette': ['green', 'yellow', 'orange']
}, 'Soil Loss')

Map

In [ ]:
# Download the total soil loss
geemap.ee_export_image(soil_loss_with_gully, filename='Total_Soil_Loss.tif', scale=90, region=choke.geometry(), file_per_band=False)

In [ ]:
# Mean soil loss for North and South Gojjam
north_gojjam = ee.FeatureCollection("projects/ee-yilkalg3/assets/Choke_watersheds/North_Gojjam")
south_gojjam = ee.FeatureCollection("projects/ee-yilkalg3/assets/Choke_watersheds/South_Gojjam")

# Mean soil loss for North Gojjam
north_soil_loss = soil_loss_with_gully.reduceRegion(
    reducer=ee.Reducer.mean(), geometry=north_gojjam, scale=90)
#print('Mean Soil Loss for North Gojjam:', north_soil_loss.getInfo())

# Mean soil loss for South Gojjam
south_soil_loss = soil_loss_with_gully.reduceRegion(
    reducer=ee.Reducer.mean(), geometry=south_gojjam, scale=90)
#print('Mean Soil Loss for South Gojjam:', south_soil_loss.getInfo())

# Severity


In [ ]:
# Reclassify total soil loss into severity classes
severity_class = (
    soil_loss_with_gully.expression(
        '(b("Soil Loss") <= 5) ? 1'
        ': (b("Soil Loss") <= 15) ? 2'
        ': (b("Soil Loss") <= 30) ? 3'
        ': (b("Soil Loss") <= 50) ? 4'
        ': 5'
    )
    .rename('Severity')
    .clip(choke)
)

In [ ]:
# Calculate the area of each severity class in square kilometers
# Class 1 (0 to 5 t/ha/year)
class_1_area_dict = severity_class.eq(1).multiply(ee.Image.pixelArea()).divide(1000000).reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=choke.geometry(),
    scale=90
)

# The key is usually 'constant' for single-band images created this way
class_1_area = class_1_area_dict.get('Severity')

#print('Area of Severity Class 1:', class_1_area.getInfo(), 'sq km')

# Class 2 (5 to 15 t/ha/year)
class_2_area_dict = severity_class.eq(2).multiply(ee.Image.pixelArea()).divide(1000000).reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=choke.geometry(),
    scale=90
)

# The key is usually 'constant' for single-band images created this way
class_2_area = class_2_area_dict.get('Severity')

#print('Area of Severity Class 2:', class_2_area.getInfo(), 'sq km')

# Class 3 (15 to 30 t/ha/year)
class_3_area_dict = severity_class.eq(3).multiply(ee.Image.pixelArea()).divide(1000000).reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=choke.geometry(),
    scale=90
)

# The key is usually 'constant' for single-band images created this way
class_3_area = class_3_area_dict.get('Severity')

#print('Area of Severity Class 3:', class_3_area.getInfo(), 'sq km')

# Class 4 (30 to 50 t/ha/year)
class_4_area_dict = severity_class.eq(4).multiply(ee.Image.pixelArea()).divide(1000000).reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=choke.geometry(),
    scale=90
)

# The key is usually 'constant' for single-band images created this way
class_4_area = class_4_area_dict.get('Severity')

#print('Area of Severity Class 4:', class_4_area.getInfo(), 'sq km')

# Class 5 (greater than 50 t/ha/year)
class_5_area_dict = severity_class.eq(5).multiply(ee.Image.pixelArea()).divide(1000000).reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=choke.geometry(),
    scale=90
)
# The key is usually 'constant' for single-band images created this way
class_5_area = class_5_area_dict.get('Severity')

#print('Area of Severity Class 5:', class_5_area.getInfo(), 'sq km')

# Zonal


In [ ]:
# Important watersheds 
Azuare = ee.FeatureCollection("projects/ee-yilkalg3/assets/Choke_watersheds/Azuare")
Birr = ee.FeatureCollection("projects/ee-yilkalg3/assets/Choke_watersheds/Birr")
Chemoga = ee.FeatureCollection("projects/ee-yilkalg3/assets/Choke_watersheds/Chemoga")
Fetam = ee.FeatureCollection("projects/ee-yilkalg3/assets/Choke_watersheds/Fetam")
Getila = ee.FeatureCollection("projects/ee-yilkalg3/assets/Choke_watersheds/Getila")
Jedeb = ee.FeatureCollection("projects/ee-yilkalg3/assets/Choke_watersheds/Jedeb")
Lah = ee.FeatureCollection("projects/ee-yilkalg3/assets/Choke_watersheds/Lah")
Muga = ee.FeatureCollection("projects/ee-yilkalg3/assets/Choke_watersheds/Muga")
Sede = ee.FeatureCollection("projects/ee-yilkalg3/assets/Choke_watersheds/Sede")
Shina = ee.FeatureCollection("projects/ee-yilkalg3/assets/Choke_watersheds/Shina")
Suha = ee.FeatureCollection("projects/ee-yilkalg3/assets/Choke_watersheds/Suha")
Temcha = ee.FeatureCollection("projects/ee-yilkalg3/assets/Choke_watersheds/Temcha")
Yeda = ee.FeatureCollection("projects/ee-yilkalg3/assets/Choke_watersheds/Yeda")

In [ ]:
azuare_mean = soil_loss_with_gully.reduceRegion(reducer=ee.Reducer.mean(), geometry=Azuare, scale=90)
#print('Azuare Mean:', azuare_mean.getInfo())

birr_mean = soil_loss_with_gully.reduceRegion(reducer=ee.Reducer.mean(), geometry=Birr, scale=90)
#print('Birr Mean:', birr_mean.getInfo())

chemoga_mean = soil_loss_with_gully.reduceRegion(reducer=ee.Reducer.mean(), geometry=Chemoga, scale=90)
#print('Chemoga Mean:', chemoga_mean.getInfo())

fetam_mean = soil_loss_with_gully.reduceRegion(reducer=ee.Reducer.mean(), geometry=Fetam, scale=90)
#print('Fetam Mean:', fetam_mean.getInfo())

getila_mean = soil_loss_with_gully.reduceRegion(reducer=ee.Reducer.mean(), geometry=Getila, scale=90)
#print('Getila Mean:', getila_mean.getInfo())

jedeb_mean = soil_loss_with_gully.reduceRegion(reducer=ee.Reducer.mean(), geometry=Jedeb, scale=90)
#print('Jedeb Mean:', jedeb_mean.getInfo())

lah_mean = soil_loss_with_gully.reduceRegion(reducer=ee.Reducer.mean(), geometry=Lah, scale=90)
#print('Lah Mean:', lah_mean.getInfo())

muga_mean = soil_loss_with_gully.reduceRegion(reducer=ee.Reducer.mean(), geometry=Muga, scale=90)
#print('Muga Mean:', muga_mean.getInfo())

sede_mean = soil_loss_with_gully.reduceRegion(reducer=ee.Reducer.mean(), geometry=Sede, scale=90)
#print('Sede Mean:', sede_mean.getInfo())

shina_mean = soil_loss_with_gully.reduceRegion(reducer=ee.Reducer.mean(), geometry=Shina, scale=90)
#print('Shina Mean:', shina_mean.getInfo())

suha_mean = soil_loss_with_gully.reduceRegion(reducer=ee.Reducer.mean(), geometry=Suha, scale=90)
#print('Suha Mean:', suha_mean.getInfo())

temcha_mean = soil_loss_with_gully.reduceRegion(reducer=ee.Reducer.mean(), geometry=Temcha, scale=90)
#print('Temcha Mean:', temcha_mean.getInfo())

yeda_mean = soil_loss_with_gully.reduceRegion(reducer=ee.Reducer.mean(), geometry=Yeda, scale=90)
#print('Yeda Mean:', yeda_mean.getInfo())

# Stats by LULC


In [ ]:
# Mapping of class codes to names
land_cover_types = {
    0: "water",
    1: "trees",
    2: "grass",
    3: "flooded_vegetation",
    4: "crops",
    5: "shrub_and_scrub",
    6: "built",
    7: "bare"
}

# Dictionary to store results
avg_soil_loss_by_class = {}

for class_code, class_name in land_cover_types.items():
    # Create a mask for the current land cover class
    class_mask = landcover.eq(class_code)
    # Mask the soil loss image
    class_soil_loss = soil_loss_with_gully.updateMask(class_mask)
    # Calculate mean soil loss for this class
    mean_dict = class_soil_loss.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=choke.geometry(),
        scale=90,  # use the appropriate scale of your analysis
        maxPixels=1e13
    )
    # Get the mean value (may need to match your band name)
    mean_soil_loss = mean_dict.get('Soil Loss')
    avg_soil_loss_by_class[class_name] = mean_soil_loss

# Print results
for class_name, mean_soil_loss in avg_soil_loss_by_class.items():
    print(f"{class_name}: {mean_soil_loss.getInfo() if mean_soil_loss else 'No data'}")

In [ ]:
districts = ee.FeatureCollection("projects/ee-yilkalg3/assets/Choke_watersheds/Districts")

# Dictionary to store results
avg_soil_loss_by_district = {}

# Get the list of district names
district_names = districts.aggregate_array('W_NAME').getInfo()

for district_name in district_names:
    # Filter the district by name
    district_fc = districts.filter(ee.Filter.eq('W_NAME', district_name))
    # Calculate mean soil loss for this district
    mean_dict = soil_loss_with_gully.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=district_fc.geometry(),
        scale=90,
        maxPixels=1e13
    )
    mean_soil_loss = mean_dict.get('Soil Loss')
    avg_soil_loss_by_district[district_name] = mean_soil_loss

# Print results
for district_name, mean_soil_loss in avg_soil_loss_by_district.items():
    print(f"{district_name}: {mean_soil_loss.getInfo() if mean_soil_loss else 'No data'}")

In [ ]:
# Get unique soil types
soil_types = soil.aggregate_array('SOIL_TYPE').distinct().getInfo()

# Dictionary to store results
avg_soil_loss_by_soil_type = {}

for soil_type in soil_types:
    # Filter soil polygons by type
    soil_fc = soil.filter(ee.Filter.eq('SOIL_TYPE', soil_type))
    # Calculate mean soil loss for this soil type
    mean_dict = soil_loss_with_gully.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=soil_fc.geometry(),
        scale=90,
        maxPixels=1e13
    )
    mean_soil_loss = mean_dict.get('Soil Loss')
    avg_soil_loss_by_soil_type[soil_type] = mean_soil_loss

# Print results
for soil_type, mean_soil_loss in avg_soil_loss_by_soil_type.items():
    print(f"{soil_type}: {mean_soil_loss.getInfo() if mean_soil_loss else 'No data'}")
